![clothing_classification](clothing_classification.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

In [159]:
# Run the cells below first

In [160]:
!pip install torchmetrics
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [161]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [162]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [163]:
# Get the number of unique labels in the training dataset
num_labels = len(train_data.classes)
print(f"Number of classes : {num_labels}")
train_data.classes

Number of classes : 10


['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [164]:
# Start coding here

# Data descovery
print(f"We have {len(train_data)} images")

We have 60000 images


# Define CNN module

In [165]:
import torch.nn as nn

class MultiClassClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Adding conv layer
        # 1 input channels (gray image)
        self.conv1 = nn.Conv2d(1, 10, kernel_size=3, padding=1)
        
        # Adding a Rectilinear Unit
       
        self.relu = nn.ReLU()
        
        # Adding a Pooling layer
        
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        #Adding flatten layer
        self.flatten = nn.Flatten()
        
        # Adding a fully connected layer for classification
        
        self.fc = nn.Linear(10 * 14 * 14, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)  
        x = self.flatten(x)
        x = self.fc(x)
        return x


# Define dataloader_train

In [166]:
dataloader_train = DataLoader(
    train_data,
    batch_size = 10,
    shuffle = True,
)

# Define Training Loop

In [167]:
MultiClassClassifier = MultiClassClassifier(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(MultiClassClassifier.parameters(), lr=0.001)

for epoch in range(2):
    for images , labels in dataloader_train:
        optimizer.zero_grad()
        predictions = MultiClassClassifier(images)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        
        predictions = predictions.tolist()

# Define dataloader_test

In [168]:
dataloader_test = DataLoader(
    test_data,
    batch_size = 10,
    shuffle = False,
)

# Define Metrics

In [169]:
from torchmetrics import Precision, Recall , Accuracy
metric_accuracy = Accuracy(task="multiclass" , num_classes=10)
metric_precision = Precision(task="multiclass", num_classes=10, average=None)
metric_recall = Recall(task="multiclass", num_classes=10, average=None)

# Define Evaluation Loop

In [170]:
MultiClassClassifier.eval()
predictions = []
with torch.no_grad():
    for images, labels in dataloader_test:
        outputs = MultiClassClassifier(images)
        _, preds = torch.max(outputs, 1)
        metric_accuracy(preds, labels)
        metric_precision(preds, labels)
        metric_recall(preds, labels)
        predictions.extend(preds.tolist())

In [171]:
len(predictions)

10000

# Compute Metrics

In [172]:
accuracy = metric_accuracy.compute().item()
precision = metric_precision.compute().tolist()
recall = metric_recall.compute().tolist()
print('Accuracy:', accuracy)
print('Precision (per class):', precision)
print('Recall (per class):', recall)

Accuracy: 0.8873999714851379
Precision (per class): [0.8113383054733276, 0.9807886481285095, 0.8432989716529846, 0.8720930218696594, 0.7951807379722595, 0.9875518679618835, 0.7571606636047363, 0.9082483649253845, 0.9332697987556458, 0.9760167002677917]
Recall (per class): [0.8730000257492065, 0.9700000286102295, 0.8180000185966492, 0.8999999761581421, 0.8579999804496765, 0.9520000219345093, 0.6079999804496765, 0.9800000190734863, 0.9789999723434448, 0.9359999895095825]
